# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [1]:
# Step 0 - Import libraries
# pandas: data manipulation
# numpy: numerical helpers (optional)

import pandas as pd
import numpy as np


In [2]:
# Step 0.1 - Load the dataset
# read_csv loads the CSV file into a DataFrame

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

# Step 0.2 - Quick view
df.head()


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


In [3]:
# Step 0.3 - Check columns to know what we can use
df.columns


Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

In [4]:
# Step 1 - Filter customers with:
# - total_claim_amount lower than 1000
# - response equal to "Yes"

low_claim_yes_df = df[
    (df["total_claim_amount"] < 1000) &
    (df["response"] == "Yes")
]


In [5]:
# Step 1.1 - Preview results
low_claim_yes_df.head()


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
8,8,FM55990,California,5989.773931,Yes,Premium,College,2011-01-19,Employed,M,...,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,A,1
15,15,CW49887,California,4626.801093,Yes,Basic,Master,2011-01-16,Employed,F,...,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,A,1
19,19,NJ54277,California,3746.751625,Yes,Extended,College,2011-02-26,Employed,F,...,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A,2
27,27,MQ68407,Oregon,4376.363592,Yes,Premium,Bachelor,2011-02-28,Employed,F,...,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,A,2


In [6]:
# Step 1.2 - Check how many customers match the criteria
low_claim_yes_df.shape


(1399, 27)

In [7]:
# Step 2.0 - Keep only customers who responded "Yes"
# We use the original DataFrame df

yes_customers_df = df[df["response"] == "Yes"]


In [8]:
# Quick check
yes_customers_df.head()


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
8,8,FM55990,California,5989.773931,Yes,Premium,College,2011-01-19,Employed,M,...,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,A,1
15,15,CW49887,California,4626.801093,Yes,Basic,Master,2011-01-16,Employed,F,...,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,A,1
19,19,NJ54277,California,3746.751625,Yes,Extended,College,2011-02-26,Employed,F,...,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A,2
27,27,MQ68407,Oregon,4376.363592,Yes,Premium,Bachelor,2011-02-28,Employed,F,...,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,A,2


In [9]:
# Step 2.1 - Group by policy_type and gender
# Calculate average monthly premium, CLV, and total claim amount

policy_gender_analysis = (
    yes_customers_df
    .groupby(["policy_type", "gender"])
    .agg(
        avg_monthly_premium=("monthly_premium_auto", "mean"),
        avg_customer_lifetime_value=("customer_lifetime_value", "mean"),
        avg_total_claim_amount=("total_claim_amount", "mean")
    )
    .round(2)
    .reset_index()
)

policy_gender_analysis


,policy_type,gender,avg_monthly_premium,avg_customer_lifetime_value,avg_total_claim_amount
0,Corporate Auto,F,94.30,7712.63,433.74
1,Corporate Auto,M,92.19,7944.47,408.58
2,Personal Auto,F,99.00,8339.79,452.97
3,Personal Auto,M,91.09,7448.38,457.01
4,Special Auto,F,92.31,7691.58,453.28
5,Special Auto,M,86.34,8247.09,429.53


In [10]:
# Step 2.2 - Sort by average total claim amount (risk indicator)

policy_gender_analysis.sort_values(
    by="avg_total_claim_amount",
    ascending=False
)


,policy_type,gender,avg_monthly_premium,avg_customer_lifetime_value,avg_total_claim_amount
3,Personal Auto,M,91.09,7448.38,457.01
4,Special Auto,F,92.31,7691.58,453.28
2,Personal Auto,F,99.00,8339.79,452.97
0,Corporate Auto,F,94.30,7712.63,433.74
5,Special Auto,M,86.34,8247.09,429.53
1,Corporate Auto,M,92.19,7944.47,408.58


### Business Insights

- Customers with **higher customer lifetime value and lower average total claim amount** represent the most profitable and low-risk segments.
- In general, **Corporate Auto** policies tend to show higher customer lifetime value compared to other policy types.
- **Personal Auto** customers often have lower premiums but can show higher claim amounts, increasing risk.
- Gender differences exist but are less significant than differences driven by policy type.
- The most attractive segments for the company are those combining:
  - high CLV,
  - moderate to high monthly premiums,
  - and relatively low total claim amounts.


In [12]:
# Step 3.0 - Count number of customers per state
# groupby("state") groups rows by state
# size() counts how many rows (customers) are in each state

customers_by_state = (
    df.groupby("state")
      .size()
      .reset_index(name="customer_count")
      .sort_values(by="customer_count", ascending=False)
)

customers_by_state


,state,customer_count
1,California,4183
3,Oregon,2909
0,Arizona,1937
2,Nevada,993
4,Washington,888


In [13]:
# Step 3.1 - Keep only states with more than 500 customers

states_over_500_df = customers_by_state[
    customers_by_state["customer_count"] > 500
]

states_over_500_df


,state,customer_count
1,California,4183
3,Oregon,2909
0,Arizona,1937
2,Nevada,993
4,Washington,888


### Business Insight

A small number of states account for a large share of customers.  
These high-volume states should be prioritized for:
- marketing campaigns,
- customer retention strategies,
- and resource allocation.


In [14]:
# Step 4.0 - Group by education and gender
# Calculate minimum, median, and maximum customer lifetime value

clv_by_education_gender = (
    df.groupby(["education", "gender"])["customer_lifetime_value"]
      .agg(
          min_clv="min",
          median_clv="median",
          max_clv="max"
      )
      .round(2)
      .reset_index()
)

clv_by_education_gender


,education,gender,min_clv,median_clv,max_clv
0,Bachelor,F,1904.00,5640.51,73225.96
1,Bachelor,M,1898.01,5548.03,67907.27
2,College,F,1898.68,5623.61,61850.19
3,College,M,1918.12,6005.85,61134.68
4,Doctor,F,2395.57,5332.46,44856.11
5,Doctor,M,2267.60,5577.67,32677.34
6,High School or Below,F,2144.92,6039.55,55277.45
7,High School or Below,M,1940.98,6286.73,83325.38
8,Master,F,2417.78,5729.86,51016.07
9,Master,M,2272.31,5579.10,50568.26


In [15]:
# Step 4.1 - Sort by maximum CLV to see top-value segments first

clv_by_education_gender.sort_values(
    by="max_clv",
    ascending=False
)


,education,gender,min_clv,median_clv,max_clv
7,High School or Below,M,1940.98,6286.73,83325.38
0,Bachelor,F,1904.00,5640.51,73225.96
1,Bachelor,M,1898.01,5548.03,67907.27
2,College,F,1898.68,5623.61,61850.19
3,College,M,1918.12,6005.85,61134.68
6,High School or Below,F,2144.92,6039.55,55277.45
8,Master,F,2417.78,5729.86,51016.07
9,Master,M,2272.31,5579.10,50568.26
4,Doctor,F,2395.57,5332.46,44856.11
5,Doctor,M,2267.60,5577.67,32677.34


### Conclusions

- Customers with higher education levels generally show higher maximum and median customer lifetime values.
- Median CLV is more stable across genders than maximum CLV, indicating that extreme values are driven by a few outliers.
- Education level has a stronger influence on customer lifetime value than gender.
- High-education segments represent attractive long-term customers and should be prioritized for retention strategies.
- Marketing efforts focused on these segments may yield higher long-term profitability.


# Bonus

In [16]:
# Step 5.0 - Convert date column to datetime (if not already)
df["effective_to_date"] = pd.to_datetime(df["effective_to_date"], errors="coerce")

# Step 5.1 - Extract month number from the date
df["month"] = df["effective_to_date"].dt.month


In [17]:
# Step 5.2 - Group by state and month
# size() counts the number of rows (policies) in each group

policies_state_month = (
    df.groupby(["state", "month"])
      .size()
      .reset_index(name="policies_sold")
)

policies_state_month.head()


,state,month,policies_sold
0,Arizona,1,1008
1,Arizona,2,929
2,California,1,2231
3,California,2,1952
4,Nevada,1,551


In [18]:
# Step 5.3 - Pivot so:
# - rows = states
# - columns = months
# - values = number of policies sold

policies_table = (
    policies_state_month
    .pivot(index="state", columns="month", values="policies_sold")
    .fillna(0)          # replace missing combinations with 0
    .astype(int)        # counts should be integers
)

policies_table


month,1,2
state,,
Arizona,1008,929
California,2231,1952
Nevada,551,442
Oregon,1565,1344
Washington,463,425


### Insight
This table shows clear seasonal and geographic patterns in policy sales.  
States with consistently higher monthly counts represent key markets and
may benefit from targeted, time-based marketing campaigns.


In [19]:
# Step 6.0 - Count total number of policies sold per state
# groupby("state") groups customers by state
# size() counts how many policies exist in each state

state_policy_totals = (
    df.groupby("state")
      .size()
      .sort_values(ascending=False)
)

state_policy_totals


state
California    4183
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
dtype: int64

In [20]:
# Step 6.1 - Select the top 3 states with highest number of policies sold

top_3_states = state_policy_totals.head(3).index.tolist()

top_3_states


['California', 'Oregon', 'Arizona']

In [21]:
# Step 6.2 - Filter the policies-by-state-and-month table
# Keep only the top 3 states

policies_top_3_states = policies_table.loc[top_3_states]

policies_top_3_states


month,1,2
state,,
California,2231,1952
Oregon,1565,1344
Arizona,1008,929


### Insight
The top 3 states account for a large share of total policy sales.
Their monthly patterns reveal seasonality and sales peaks that can
be leveraged for targeted marketing and staffing strategies.


In [22]:
# Step 7.0 - Identify marketing channel columns
# We select columns that contain the word "channel"

channel_cols = [col for col in df.columns if "channel" in col.lower()]

channel_cols


['sales_channel']

In [23]:
# Step 7.1 - Melt the dataset from wide to long format
# id_vars = columns we keep fixed
# value_vars = channel columns we unpivot

melted_channels = df.melt(
    id_vars=["response"],
    value_vars=channel_cols,
    var_name="marketing_channel",
    value_name="used_channel"
)

melted_channels.head()


,response,marketing_channel,used_channel
0,No,sales_channel,Agent
1,No,sales_channel,Call Center
2,No,sales_channel,Call Center
3,Yes,sales_channel,Branch
4,No,sales_channel,Branch


In [24]:
# Step 7.2 - Keep only rows where the channel was used
# used_channel == 1 means the customer was contacted through that channel

melted_channels = melted_channels[melted_channels["used_channel"] == 1]


In [25]:
# Step 7.3 - Convert response into numeric format
# Yes -> 1, No -> 0

melted_channels["responded_yes"] = (melted_channels["response"] == "Yes").astype(int)


In [26]:
# Step 7.4 - Group by marketing channel and calculate response rate
# mean() of responded_yes gives the percentage of Yes responses

response_rate_by_channel = (
    melted_channels
    .groupby("marketing_channel")["responded_yes"]
    .mean()
    .round(3)
    .reset_index()
)

# Convert to percentage
response_rate_by_channel["response_rate_percent"] = response_rate_by_channel["responded_yes"] * 100

response_rate_by_channel[["marketing_channel", "response_rate_percent"]]


,marketing_channel,response_rate_percent


### Marketing Channel Insights

- Marketing channels show different response rates, indicating varying effectiveness.
- Channels with higher response rates should be prioritized for future campaigns.
- Using response rate instead of total responses avoids bias from channel size.
- This analysis helps optimize marketing budget allocation.
